In [2]:
import pandas as pd
import numpy as np

import math
import sqlite3
import os

In [10]:
# define path for database file, you will have to edit to your repository path or it will NOT work
path = "C:\\Users\\scott\\github\\OCCE_HVAC_Comparison\\"
filename = "NEAA_final_dataset.db"

# make connection
con = sqlite3.connect(os.path.join(path, filename))

In [11]:
t_oa = 14

t_stpt = 72 # room setpoint temperatures

In [12]:
# text for OA sql query
sql_query = "SELECT siteid, ahuid, systemid, hvaczoneid, hashrverv, " #
sql_query += "cfm, equiptype, qty, minimumoutsideair FROM AHUS" #

OA_HEATING = pd.read_sql_query(sql_query, con)

In [13]:
OA_HEATING.head()

,siteid,ahuid,systemid,hvaczoneid,hashrverv,cfm,equiptype,qty,minimumoutsideair
0,100016,1,1,1.0,Yes,3000,Gas Pack with AC,1.0,3000
1,100016,2,1,1.0,Yes,3000,Gas Pack with AC,1.0,3000
2,100201,1,None,1.0,No,2000,Gas Pack with AC,3.0,
3,100201,2,None,1.0,No,2500,Gas Pack with AC,6.0,
4,100201,3,None,1.0,No,1200,Gas Pack with AC,2.0,


In [17]:
#sum cfms for the site
#OA_cfm = OutdoorAir.groupby('siteid').sum()
OA_HEATING = OA_HEATING.replace(r'^\s*$', np.nan, regex=True) #replace blank spaces with NaN

flag = []

#set flag to track which have no OA input
i=0
for i in range(0, len(OA_HEATING)):
    if math.isnan(OA_HEATING['minimumoutsideair'][i]) == True:
        flag.append(1)
    else:
        flag.append(0)
        
#calculate OA fraction
OA_HEATING['OutdoorAirFraction'] = OA_HEATING['minimumoutsideair'] / OA_HEATING['cfm']

In [20]:
#calculate AHU entering air temperature at design OA temp
OA_HEATING['AHU EAT [°F]'] = OA_HEATING['OutdoorAirFraction']*t_oa + (1 - OA_HEATING['OutdoorAirFraction'])*t_stpt

OA_HEATING['Q_OA [btuh]'] = (1.08 * OA_HEATING['cfm'] * (t_stpt - OA_HEATING['AHU EAT [°F]'])) * OA_HEATING['qty']

OA_HEATING = OA_HEATING.groupby('siteid').sum()

In [21]:
OA_HEATING.head()

,ahuid,hvaczoneid,cfm,qty,minimumoutsideair,OutdoorAirFraction,AHU EAT [°F],Q_OA [btuh]
siteid,,,,,,,,
100016,3,2.0,6000.0,2.0,6000.0,2.000000,28.000000,375840.0
100201,6,3.0,5700.0,11.0,0.0,0.000000,0.000000,0.0
101127,3,4.0,3200.0,2.0,3200.0,2.000000,28.000000,200448.0
101959,1,1.0,15000.0,2.0,0.0,0.000000,0.000000,0.0
102217,21,6.0,20800.0,22.0,3255.0,0.793333,385.986667,1019152.8
